In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Crawling Link Dokumen

In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.Vme9hKouJI/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.dhv5wWeSCi/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.TgyJJ8RTIU/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:3 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:12 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Get:13 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [620 kB]
Hit:14 ht

In [ ]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pickle

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1300, 800")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
def collecting_repository(URL, num_page, file_name, prodi):
  print("collecting all repository from prodi {}".format(prodi))
  print("consists of {} page(s)".format(num_page))
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - -")
  # mendefinisikan tempat untuk menyimpan link repository
  link_repository = []
  # menghubungkan dengan driver chrome
  driver = web_driver()
  driver.get(URL)
  time.sleep(10)
  driver.save_screenshot("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Screenshot/{}_screenshot_page_1.png".format(prodi))
  # menjalankan untuk tiap halaman web
  for page in range(num_page):
  # mengambil konten web
    content = driver.page_source
    # membuat parsing html
    soup = BeautifulSoup(content, "html.parser")
    results = soup.find(id="example_wrapper")
    repository_elements = results.find_all("div", class_="col-lg-12 col-md-12 col-sm-12 col-xs-12")
    for repository_element in repository_elements:
      link = repository_element.find("a")['href']
      link_repository.append(link)
    if len(link_repository) > (page)*10:
      print("collecting repository page-{} succeed...".format(page+1))
    # menuju halaman web selanjutnya
    if page < num_page-1:
      next_click = driver.find_element("link text", "Next")
      next_click.click()
      time.sleep(10)
      driver.save_screenshot("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Screenshot/{}_screenshot_page_{}.png".format(prodi, page+2))
  driver.quit()
  # menyimpan list berisi link repository
  open_file = open("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/{}_".format(prodi)+file_name, "wb")
  pickle.dump(link_repository, open_file)
  open_file.close()
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - -")
  print("done collecting all repository from page-1 to page-{}".format(page+1))
  print("total {} repository".format(len(link_repository)))
  print("-----------------------------------------------------")
  print("repository will save to {}".format("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/{}_".format(prodi)+file_name))

#### Prodi 5262 (S1 STATISTIKA) -> 449 repository

In [ ]:
prodi = "5262" # S1 Statistika
URL = "http://etd.repository.ugm.ac.id/home/result_pencarianlanjut/search?key_one=penJudul&keyword_one=a&logic_two=AND&key_two=penJudul&keyword_two=&logic_three=AND&key_three=penJudul&keyword_three=&tahunawal=&tahunakhir=&jeniskarya=&prodi={}".format(prodi)
num_page = 45
file_name = "link_repository"
collecting_repository(URL, num_page, file_name, prodi)

collecting all repository from prodi 5262
consists of 45 page(s)
- - - - - - - - - - - - - - - - - - - - - - - - - - -
collecting repository page-1 succeed...
collecting repository page-2 succeed...
collecting repository page-3 succeed...
collecting repository page-4 succeed...
collecting repository page-5 succeed...
collecting repository page-6 succeed...
collecting repository page-7 succeed...
collecting repository page-8 succeed...
collecting repository page-9 succeed...
collecting repository page-10 succeed...
collecting repository page-11 succeed...
collecting repository page-12 succeed...
collecting repository page-13 succeed...
collecting repository page-14 succeed...
collecting repository page-15 succeed...
collecting repository page-16 succeed...
collecting repository page-17 succeed...
collecting repository page-18 succeed...
collecting repository page-19 succeed...
collecting repository page-20 succeed...
collecting repository page-21 succeed...
collecting repository page-22

In [ ]:
open_file = open("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/5262_link_repository", "rb")
link_repository_5262 = pickle.load(open_file)
open_file.close()

In [ ]:
link_repository_5262

['http://etd.repository.ugm.ac.id/home/detail_pencarian/209155',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/214787',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/215812',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/210706',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/212500',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/210968',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/210714',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/214308',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/214822',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/217146',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/211789',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/211304',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/213097',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/217967',
 'http://etd.repository.ugm.ac.id/home/detail_pencarian/211569',
 'http://etd.repository.u

In [ ]:
len(link_repository_5262)

449

#### Menggabungkan semua link_repository

In [ ]:
link_repository = link_repository_5262

In [ ]:
set([x for x in link_repository if link_repository.count(x) > 1])

set()

In [ ]:
len(link_repository)

In [ ]:
open_file = open("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/all_link_repository", "wb")
pickle.dump(link_repository, open_file)
open_file.close()

In [ ]:
open_file = open("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/all_link_repository", "rb")
link_repository = pickle.load(open_file)
open_file.close()

In [ ]:
link_repository

### Scraping Per Judul

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def collecting_documents(list_link):
  print("collecting all documents from {} repository".format(len(list_link)))
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - -")
  k = 1
  # mendefinisikan tempat untuk menyimpan dokumen tugas akhir
  titles = []
  authors = []
  years = []
  abstracts_in = []
  abstracts_en = []
  keywords = []
  all_documents = [titles, authors, years, abstracts_in, abstracts_en, keywords]
  name_documents = ['titles', 'authors', 'years', 'abstracts_in', 'abstracts_en', 'keywords']
  # membuat header untuk kebutuhan user agent pada request
  headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
  # menjalankan untuk tiap repository
  for rep in list_link:
    # membuat request URL
    URL = rep
    page = requests.get(URL, headers)
    # membuat parsing html
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="body")
    document_elements = results.find_all("div", class_="container")
    # mengambil data sesuai kebutuhan (judul, pengarang, tahun, abstrak_in, abstrak_en, kata_kunci)
    for document_element in document_elements:
      title = document_element.find("p", class_="title").text.strip()
      author = document_element.find("p", class_="penulis").text.strip()
      year = document_element.find("strong", class_="detail-karya-akhir").text.strip()
      abstract_in = document_element.find("p", class_="abstrak").text.strip()
      abstract_en = document_element.find("p", class_="abstrak").findNext('p').text.strip()
      keyword = document_element.find("p", class_="keyword").text.strip()
      # menggabungkan seluruh data
      titles.append(title)
      authors.append(author)
      years.append(year)
      abstracts_in.append(abstract_in)
      abstracts_en.append(abstract_en)
      keywords.append(keyword)
    if k % 10 == 0:
      print("collecting {} documents succeed...".format(k))
    k += 1
  # menyimpan list berisi judul, pengarang, tahun, abstrak_in, abstrak_en, kata_kunci
  j = 0
  for docs in all_documents:
    open_file = open("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/all_"+name_documents[j], "wb")
    pickle.dump(docs, open_file)
    open_file.close()
    j += 1
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - -")
  print("done collecting all documents from {} repository".format(len(list_link)))
  print("total {} documents titles".format(len(titles)))
  print("total {} documents authors".format(len(authors)))
  print("total {} documents years".format(len(years)))
  print("total {} documents abstracts_in".format(len(abstracts_in)))
  print("total {} documents abstracts_en".format(len(abstracts_en)))
  print("total {} documents keywords".format(len(keywords)))
  print("-----------------------------------------------------")
  for docs in name_documents:
    print("documents {} will save to {}".format(docs, "/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/all_"+docs))

In [ ]:
list_link = link_repository.copy()
collecting_documents(list_link)

collecting all documents from 770 repository
- - - - - - - - - - - - - - - - - - - - - - - - - - -
collecting 10 documents succeed...
collecting 20 documents succeed...
collecting 30 documents succeed...
collecting 40 documents succeed...
collecting 50 documents succeed...
collecting 60 documents succeed...
collecting 70 documents succeed...
collecting 80 documents succeed...
collecting 90 documents succeed...
collecting 100 documents succeed...
collecting 110 documents succeed...
collecting 120 documents succeed...
collecting 130 documents succeed...
collecting 140 documents succeed...
collecting 150 documents succeed...
collecting 160 documents succeed...
collecting 170 documents succeed...
collecting 180 documents succeed...
collecting 190 documents succeed...
collecting 200 documents succeed...
collecting 210 documents succeed...
collecting 220 documents succeed...
collecting 230 documents succeed...
collecting 240 documents succeed...
collecting 250 documents succeed...
collecting

In [ ]:
open_file = open("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/all_titles", "rb")
titles = pickle.load(open_file)
open_file.close()
titles = [x.replace("\r\n"," ") for x in titles]
titles = [x.replace("\n"," ") for x in titles]
titles

In [ ]:
open_file = open("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/all_authors", "rb")
authors = pickle.load(open_file)
open_file.close()
authors = [x.replace("\r\n"," ") for x in authors]
authors = [x.replace("\n"," ") for x in authors]
authors

In [ ]:
open_file = open("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/all_years", "rb")
years = pickle.load(open_file)
open_file.close()
years = [x.replace("\r\n"," ") for x in years]
years = [x.replace("\n"," ") for x in years]
years

In [ ]:
open_file = open("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/all_abstracts_in", "rb")
abstracts_in = pickle.load(open_file)
open_file.close()
abstracts_in = [x.replace("\r\n"," ") for x in abstracts_in]
abstracts_in = [x.replace("\n"," ") for x in abstracts_in]
abstracts_in

In [ ]:
open_file = open("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/all_abstracts_en", "rb")
abstracts_en = pickle.load(open_file)
open_file.close()
abstracts_en = [x.replace("\r\n"," ") for x in abstracts_en]
abstracts_en = [x.replace("\n"," ") for x in abstracts_en]
abstracts_en

In [ ]:
open_file = open("/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/all_keywords", "rb")
keywords = pickle.load(open_file)
open_file.close()
keywords = [x.replace("\r\n"," ") for x in keywords]
keywords = [x.replace("\n"," ") for x in keywords]
keywords

### Memindahkan ke Dataframe

In [ ]:
import pandas as pd

In [ ]:
documents_thesis = pd.DataFrame(data={'link_repository':link_repository, 'titles':titles, 'authors':authors, 'years':years, 'abstracts_in':abstracts_in, 'abstracts_en':abstracts_en, 'keywords':keywords})

In [ ]:
documents_thesis

,link_repository,titles,authors,years,abstracts_in,abstracts_en,keywords
0,http://etd.repository.ugm.ac.id/home/detail_pe...,Model Pohon Regresi dengan Pendekatan Kredibil...,"AULADIENA SHAUMU RODHIYATIN, Danang Teguh Qoyy...",2022 | Skripsi | S1 STATISTIKA,Dalam perhitungan premi terdapat salah satu me...,One of the methods used in premium calculation...,"Kata Kunci : premi, teori kredibilitas, BÃƒÂ¼h..."
1,http://etd.repository.ugm.ac.id/home/detail_pe...,PEMODELAN TOPIK PADA DATA REVIEW MENGGUNAKAN T...,"ARUNG RIZKY ANENURAT, Drs. Danardono, MPH., Ph.D.",2022 | Skripsi | S1 STATISTIKA,Dalam upaya membatasi pertumbuhan Covid-19 den...,In an effort to limit the growth of Covid-19 b...,"Kata Kunci : Pemodelan topik, Latent Dirichlet..."
2,http://etd.repository.ugm.ac.id/home/detail_pe...,ANALISIS CHURN PADA PELANGGAN PRODUK SUSU MENG...,"MOCHAMAD RAIHAN M, Prof. Dr.rer.nat. Dedi Rosa...",2022 | Skripsi | S1 STATISTIKA,Dalam menjalankan bisnis hampir di seluruh ind...,"In running a business, almost all industries h...","Kata Kunci : Customer Churn, Backpropagation, ..."
3,http://etd.repository.ugm.ac.id/home/detail_pe...,Aplikasi XGBoost pada Prediksi Cadangan Klaim ...,"RUTH CORNELIA N, Danang Teguh Qoyyimi, M.Sc., ...",2022 | Skripsi | S1 STATISTIKA,Semua kegiatan manusia tidak terlepas dari ris...,All human activities are inseparable from risk...,"Kata Kunci : Cadangan Klaim, Chain-Ladder, Ext..."
4,http://etd.repository.ugm.ac.id/home/detail_pe...,VALUASI OBLIGASI DENGAN PEMODELAN SUKU BUNGA C...,"PUTRI DWI RAMADHANI, Dr.Abdurakhman, S.Si., M.Si.",2022 | Skripsi | S1 STATISTIKA,Obligasi merupakan surat hutang jangka panjang...,Bonds are financial instruments in the form of...,"Kata Kunci : Risiko Kredit, Model Merton, Suku..."
...,...,...,...,...,...,...,...
765,http://etd.repository.ugm.ac.id/home/detail_pe...,DISTRIBUSI MAJEMUK DENGAN JUMLAHAN BOREL DAN F...,"I Gusti Ngurah Putra Pratama, Adhitya Ronnie E...",2015 | Skripsi | PROGRAM STUDI S1 STATISTIKA,Total besar klaim asuransi adalah jumlah dari ...,,"Kata Kunci : distribusi Borel, distribusi maje..."
766,http://etd.repository.ugm.ac.id/home/detail_pe...,KLASIFIKASI SAHAM BERDASARKAN MODEL GARCH MENG...,"ARSITA FAJRIN KHOIRUNISA, Adhitya Ronnie Effendie",2012 | Skripsi | PROGRAM STUDI S1 STATISTIKA,Ekstensifikasi maupun intensifikasi pengembang...,,"Kata Kunci : Model GARCH, Analisis Klaster, Al..."
767,http://etd.repository.ugm.ac.id/home/detail_pe...,PENYELESAIAN POHON REGRESI CART (CLASSIFICATIO...,"ENDANG RAHAYU LESTARI, Danang Teguh Qoyyimi",2011 | Skripsi | PROGRAM STUDI S1 STATISTIKA,Bentuk pencarian stokastik suatu analisis klas...,,"Kata Kunci : Metode Bayesian, classification t..."
768,http://etd.repository.ugm.ac.id/home/detail_pe...,MODEL REGRESI BETA; BETA REGRESSION MODEL,"IRNA NOVITA LESTARI, Sri Haryatmi Kartiko",2012 | Skripsi | PROGRAM STUDI STATISTIK,Regresi linier klasik seringkali digunakan unt...,,"Kata Kunci : regresi beta, kontinu, distribusi..."


In [ ]:
documents_thesis.to_csv('/content/drive/MyDrive/Tugas Akhir/Data Scraping/Data/documents_thesis.csv', index=False)